In [250]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.notebook_repr_html", False)  # disable "rich" output
plt.style.use("seaborn")  # plot style

sns.set_palette([  # the "R4" palette
    "#000000", "#DF536B", "#61D04F", "#2297E6",
    "#28E2E5", "#CD0BBC", "#F5C710", "#999999"
])

plt.rcParams.update({
    "font.size":         11,
    "font.family":       "sans-serif",
    "font.sans-serif":   ["Alegreya Sans", "Alegreya"],
    "figure.autolayout": True,
    "figure.dpi":        300,
    "figure.figsize":    (6, 3.5),  # default is [8.0, 5.5],
})

In [251]:
import os, os.path
import sqlite3
baza = os.path.join('main.db')

In [252]:
Posts = pd.read_csv("travel_stackexchange_com/Posts.csv.gz", compression='gzip')
Votes = pd.read_csv("travel_stackexchange_com/Votes.csv.gz", compression='gzip')
Badges = pd.read_csv("travel_stackexchange_com/Badges.csv.gz", compression='gzip')
Users = pd.read_csv("travel_stackexchange_com/Users.csv.gz", compression='gzip')

## Zadanie 1
Top 10 etykiet Badge o najwiekszej ilosci wystąpień, z nazwą etykiety i najmniejszą klasą spośród etykiet jednego typu.

In [253]:
if os.path.isfile(baza):
    os.remove(baza)
conn = sqlite3.connect(baza)

Posts.to_sql("Posts", conn)
Badges.to_sql("Badges", conn)

resultSQL = pd.read_sql_query(
"""SELECT Name,
  COUNT(*) AS Number,
  MIN(Class) AS BestClass
  FROM Badges
  GROUP BY Name
  ORDER BY Number DESC
  LIMIT 10
""", conn)
conn.close()

In [254]:
res = Badges.loc[:, ("Id", "Name", "Class")]
res = res.groupby(["Name"]).agg(Number=('Name', 'count'), 
                               BestClass=('Class', 'min'))
res = res.sort_values(by=['Number'], ascending=False).head(10)
resultPd = res.reset_index()

In [255]:
resultSQL.equals(resultPd)

True

## Zadanie 2
Liczba postów, zamieszczonych przez wszystkich uzytkownikow pochodzących z danej lokacji.

In [256]:
if os.path.isfile(baza):
    os.remove(baza)
conn = sqlite3.connect(baza)

Posts.to_sql("Posts", conn)
Users.to_sql("Users", conn)

resultSQL = pd.read_sql_query(
"""SELECT Location, COUNT(*) AS Count
  FROM (
        SELECT Posts.OwnerUserId, Users.Id, Users.Location
        FROM Users
        JOIN Posts ON Users.Id = Posts.OwnerUserId
        )
  WHERE Location NOT IN ('')
  GROUP BY Location
  ORDER BY Count DESC
  LIMIT 10
""", conn)
conn.close()

In [257]:
res = pd.merge( how = 'inner',
    left=Users[['Id', 'Location']], right=Posts[["OwnerUserId"]], 
    left_on="Id", right_on="OwnerUserId")
res = res[['Id', 'Location']]
res = res[pd.notna(res['Location'])]
res = res.groupby(['Location']).agg(Count=('Id', 'count'))
res = res.sort_values(by=['Count'], ascending=False).head(10)
resultPd = res.reset_index()

In [258]:
resultSQL.equals(resultPd)

True

## Zadanie 3
Top 10 postów, które uzyskały najwięcej odpowiedzi.

In [259]:
if os.path.isfile(baza):
    os.remove(baza)
conn = sqlite3.connect(baza)

Posts.to_sql("Posts", conn)
Users.to_sql("Users", conn)

resultSQL = pd.read_sql_query(
"""SELECT
        Users.AccountId,
        Users.DisplayName,
        Users.Location,
        AVG(PostAuth.AnswersCount) as AverageAnswersCount
    FROM
        (
        SELECT
        AnsCount.AnswersCount,
        Posts.Id,
        Posts.OwnerUserId
        FROM 
            (
            SELECT Posts.ParentId, COUNT(*) AS AnswersCount
            FROM Posts
            WHERE Posts.PostTypeId = 2
            GROUP BY Posts.ParentId
            ) AS AnsCount
        JOIN Posts ON Posts.Id = AnsCount.ParentId
        ) AS PostAuth
    JOIN Users ON Users.AccountId=PostAuth.OwnerUserId
    GROUP BY OwnerUserId
    ORDER BY AverageAnswersCount DESC
    LIMIT 10
""", conn)
conn.close()

In [260]:
res = Posts[Posts.PostTypeId == 2][["Id", "ParentId", "PostTypeId"]]
res = res.groupby(["ParentId"]).agg(AnswersCount=('Id', 'count'))
res = pd.merge(how='inner',
    left=Posts[["Id", "OwnerUserId"]], right=res, 
    left_on="Id", right_on="ParentId")
res = pd.merge(how='inner',
    left=Users[["AccountId"]], right=res,
    left_on="AccountId", right_on="OwnerUserId")
res = res[["AccountId", "AnswersCount"]]
res = res.groupby("AccountId").agg(AverageAnswersCount=('AnswersCount', 'mean'))
res = pd.merge(how="inner",
    left=Users[["AccountId", "DisplayName", "Location"]], right=res,
    on='AccountId')
# sortowanie przez drugą kolumne "AccountId", żeby kolejność wierszy zzgadzłą się z wersją sql
res = res.sort_values(by=['AverageAnswersCount', "AccountId"], ascending=False).head(10)
res = res.reset_index(drop=True)
resultPd = res

In [261]:
resultSQL.equals(resultPd)

True

## Zadanie 4
Post(typu PostTypeId=1), który w danym roku ma najwięcej UpVote-ów (typu VoteTypeId=2). Wiersze zawierają tytuł rekordowego posta, rok zamieszenia i ilość UpVote-ów (maksimum danego roku).

In [262]:
if os.path.isfile(baza):
    os.remove(baza)
conn = sqlite3.connect(baza)

Posts.to_sql("Posts", conn)
Votes.to_sql("Votes", conn)

resultSQL = pd.read_sql_query(
"""SELECT
        Posts.Title,
        UpVotesPerYear.Year,
        MAX(UpVotesPerYear.Count) AS Count
    FROM 
        (
        SELECT
            PostId,
            COUNT(*) AS Count,
            STRFTIME('%Y', Votes.CreationDate) AS Year
        FROM Votes
        WHERE VoteTypeId=2
        GROUP BY PostId, Year
        ) AS UpVotesPerYear
    JOIN Posts ON Posts.Id=UpVotesPerYear.PostId
    WHERE Posts.PostTypeId=1
    GROUP BY Year
    ORDER BY Year ASC
""", conn)
conn.close()

In [263]:
res = Votes[Votes.VoteTypeId == 2][["PostId", "Id", "CreationDate"]]
res["CreationDate"] = pd.to_datetime(res['CreationDate'],format='%Y-%m-%d')
# astypes applied to Year column for type alignment with SQL version
res["Year"] = pd.DatetimeIndex(res['CreationDate']).year\
    .astype('str').astype(object)
res = res.groupby(["PostId", "Year"]).agg(Count=('Id', 'count'))
res = res.reset_index()
res = pd.merge( how='inner',
    left = Posts[Posts.PostTypeId == 1][["Id", "Title"]], right=res,
    left_on='Id', right_on='PostId')
# aggregation with max(Count) without dropping other columns
res = res.sort_values('Count',ascending=False)
res = res.drop_duplicates(['Year']).sort_index()

res = res[['Title', 'Year', 'Count']]
resultPd = res.reset_index(drop=True)

In [264]:
resultSQL.equals(resultPd)

True

## Zadanie 5
Top 10 postów (typu PostTypeId=1), które w latach 2021-2020 uzyskały 0 głosów. Sortowanie względem ilości głosów sprzed 2020 roku.

In [265]:
if os.path.isfile(baza):
    os.remove(baza)
conn = sqlite3.connect(baza)

Posts.to_sql("Posts", conn)
Votes.to_sql("Votes", conn)

resultSQL = pd.read_sql_query(
"""SELECT
    Posts.Title,
    VotesByAge2.OldVotes
    FROM Posts
    JOIN (
        SELECT
            PostId,
            MAX(CASE WHEN VoteDate = 'new' THEN Total ELSE 0 END) NewVotes,
            MAX(CASE WHEN VoteDate = 'old' THEN Total ELSE 0 END) OldVotes,
            SUM(Total) AS Votes
        FROM (
            SELECT
              PostId,
              CASE STRFTIME('%Y', CreationDate)
                WHEN '2021' THEN 'new'
                WHEN '2020' THEN 'new'
                ELSE 'old'
              END VoteDate,
              COUNT(*) AS Total
            FROM Votes
            WHERE VoteTypeId IN (1, 2, 5)
            GROUP BY PostId, VoteDate
            ) AS VotesByAge
        GROUP BY VotesByAge.PostId
        HAVING NewVotes=0
        ) AS VotesByAge2 ON VotesByAge2.PostId=Posts.ID
    WHERE Posts.PostTypeId=1
    ORDER BY VotesByAge2.OldVotes DESC
    LIMIT 10
""", conn)
conn.close()

In [266]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [267]:
res = Votes[Votes.VoteTypeId.isin([1, 2, 5])]
res["CreationDate"] = pd.to_datetime(res['CreationDate'],format='%Y-%m-%d')
res["Year"] = pd.DatetimeIndex(res['CreationDate']).year
res['VoteDate'] = np.where(res['Year'] == 2021, 'NewVotes',
                  np.where(res['Year'] == 2020, 'NewVotes', 'OldVotes'))
res = res.groupby(['PostId', 'VoteDate']).agg(Total=('Id', 'count'))
res = res.reset_index()
res = res.pivot(index='PostId', columns='VoteDate', values='Total')
res['NewVotes'] = res['NewVotes'].fillna(0)
res['OldVotes'] = res['OldVotes'].fillna(0)
res = res.groupby('PostId').agg(NewVotes=('NewVotes', 'max'),
                                OldVotes=('OldVotes', 'max'))
res['Votes'] = np.add(res.NewVotes, res.OldVotes)
res = res[res.NewVotes == 0]
res = pd.merge(how='inner',
    left=Posts[Posts.PostTypeId == 1][['Id', 'Title']], right=res,
    left_on='Id', right_on='PostId')
# sorting by title for complience with sql version
res = res.sort_values(by=["OldVotes", 'Title'], ascending=False).head(10)
res = res[['Title', 'OldVotes']]
res['OldVotes'] = res['OldVotes'].astype('int64')
res = res.reset_index(drop=True)
resultPd = res

In [268]:
resultSQL.equals(resultPd)

True